# ICLR 2025 OpenReview data collection

Uses OpenReview **API v2** (api2.openreview.net). Single bulk fetch via invitation—no per-paper forum calls, so collection is fast (no rate-limit bottlenecks).

In [1]:
# Install if needed (uncomment)
# !pip install openreview-py "urllib3<2.0"

import openreview
import pandas as pd
from pathlib import Path

OUTPUT_DIR = Path("")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [2]:
def _flatten_content(content):
    """API v2 content uses {field: {value: ...}}. Extract values."""
    return {k: v.get("value", v) if isinstance(v, dict) else v for k, v in content.items()}


def get_iclr_2025_submissions():
    """Fetch all ICLR 2025 submissions in one bulk call via API v2."""
    client = openreview.api.OpenReviewClient(baseurl="https://api2.openreview.net")

    invitation_id = "ICLR.cc/2025/Conference/-/Submission"
    print(f"Collecting submissions from {invitation_id}...")

    submissions = client.get_all_notes(invitation=invitation_id)
    print(f"Successfully collected {len(submissions)} submissions.")

    for i, note in enumerate(submissions[:5]):
        title = note.content.get("title", {}).get("value", "No Title")
        print(f"  {i+1}. {title}")

    return submissions

In [3]:
submissions = get_iclr_2025_submissions()

df = pd.DataFrame([_flatten_content(n.content) for n in submissions])
out_path = OUTPUT_DIR / "iclr2025_submissions.csv"
df.to_csv(out_path, index=False)
print(f"Saved {len(df)} rows to {out_path}")

Retrying request: GET /notes?invitation=ICLR.cc%2F2025%2FConference%2F-%2FSubmission&stream=true, response: {'name': 'RateLimitError', 'message': 'Too many requests: You have made 63 requests, surpassing the limit of 60 requests. Please try again in 2 seconds (2026-02-15-6761393)', 'status': 429, 'details': {'limit': 60, 'remaining': 0, 'resetTime': '2026-02-15T18:32:33.819Z', 'used': 63, 'current': 63, 'reqId': '2026-02-15-6761393'}}, error: None
Successfully collected 11672 submissions.
  1. Neuroacoustic Patterns: Constant Q Cepstral Coefficients for the Classification of Neurodegenerative Disorders
  2. A Feature-Aware Federated Learning Framework for Unsupervised Anomaly Detection in 5G Networks
  3. UnoLoRA: Single Low-Rank Adaptation for Efficient Multitask Fine-tuning
  4. Synergistic Approach for Simultaneous Optimization of Monolingual, Cross-lingual, and Multilingual Information Retrieval
  5. EXecution-Eval: Can language models execute real-world code?
Saved 11672 rows to i